In [0]:
%pip install azure-cosmos

Python interpreter will be restarted.
Collecting azure-cosmos
 Downloading azure_cosmos-4.2.0-py2.py3-none-any.whl (136 kB)
Collecting azure-core<2.0.0,>=1.0.0
 Downloading azure_core-1.15.0-py2.py3-none-any.whl (138 kB)
Requirement already satisfied: six>=1.6 in /usr/local/lib/python3.8/site-packages (from azure-cosmos) (1.15.0)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core<2.0.0,>=1.0.0->azure-cosmos) (2.24.0)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.0.0->azure-cosmos) (3.0.4)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.0.0->azure-cosmos) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.0.0->azure-cosmos) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core<2.0.0,>=1.0.0->azure-cosmos) (2020.12.5)
Installing collected packages: azure-core, azure-cosmos
Successfully installed azure-core-1.15.0 azure-cosmos-4.2.0
WARNING: You are using pip version 20.2.4; however, version 21.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-54058cc7-df95-4731-a99d-8bdc0a25c76e/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting haversine
 Downloading haversine-2.3.1-py2.py3-none-any.whl (5.5 kB)
Installing collected packages: haversine
Successfully installed haversine-2.3.1
WARNING: You are using pip version 20.2.4; however, version 21.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-54058cc7-df95-4731-a99d-8bdc0a25c76e/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import linregress
import os

from azure.cosmos import exceptions, CosmosClient, PartitionKey
# Initialize the Cosmos client
endpoint = os.environ.get('COSMOS_ENDPOINT')
key = os.environ.get('COSMOS_KEY')
outlet_id = "1"
city = "pune"

In [0]:
# Get Outlet from cosmos db
client = CosmosClient(endpoint, key)
database_name = 'bhoojal_outlets'
container_name = 'outlet'

database = client.create_database_if_not_exists(id=database_name)

container = database.create_container_if_not_exists(
    id=container_name, 
    partition_key=PartitionKey(path="/city"),
    offer_throughput=400
)

outlet_item = container.read_item(item=outlet_id, partition_key=city)
outlet_item

Out[134]: {'id': '1',
 'city': 'pune',
 'quantity_score': -0.1,
 'consumption_quota': 10000,
 'consumption_quota_available': 10000,
 'quality_score': -0.1,
 'quality_ph': -0.1,
 'quality_hardness': -0.1,
 'quality_updatedon': '',
 'location': {'type': 'Point', 'coordinates': [73.7866902, 18.5558075]},
 '_rid': 'QJplAIr5B+MBAAAAAAAAAA==',
 '_self': 'dbs/QJplAA==/colls/QJplAIr5B+M=/docs/QJplAIr5B+MBAAAAAAAAAA==/',
 '_etag': '"a3007b8b-0000-0d00-0000-60d5ee7e0000"',
 '_attachments': 'attachments/',
 '_ts': 1624632958}

In [0]:
# Query depth data from cosmosdb
# outlet_item['location']['coordinates'][0] = 73.1117764
# outlet_item['location']['coordinates'][1] = 18.9093286
container_depth = database.create_container_if_not_exists(
    id="region_depth", 
    partition_key=PartitionKey(path="/city"),
    offer_throughput=400
)
distance_query = "ST_DISTANCE(f.boundary, {\"type\":\"Point\",\"coordinates\": [" + str(outlet_item['location']['coordinates'][0]) + ", " + str(outlet_item['location']['coordinates'][1]) + "]})"
depth_query = "SELECT f.id,f.depth,f.scannedIn,"+ distance_query +" as distance FROM f WHERE " + distance_query + " < 600" 
depth_query

Out[135]: 'SELECT f.id,f.depth,f.scannedIn,ST_DISTANCE(f.boundary, {"type":"Point","coordinates": [73.7866902, 18.5558075]}) as distance FROM f WHERE ST_DISTANCE(f.boundary, {"type":"Point","coordinates": [73.7866902, 18.5558075]}) < 600'

In [0]:
depth_result = list(container_depth.query_items(
    query=depth_query,
    enable_cross_partition_query=True
))
depth_result

Out[136]: [{'id': '47890177-f72f-4b94-85b0-d3f1d645fbbb',
 'depth': 3.4,
 'scannedIn': '2020Q1',
 'distance': 585.55522363821},
 {'id': 'rohantest', 'depth': 8, 'scannedIn': '2020Q1', 'distance': 0},
 {'id': '47890177-f72f-4b94-85b0-d3f1d645fbcc',
 'depth': 4,
 'scannedIn': '2020Q2',
 'distance': 585.55522363821},
 {'id': '47890177-f72f-4b94-85b0-d3f1d645fbdd',
 'depth': 6,
 'scannedIn': '2020Q3',
 'distance': 585.55522363821},
 {'id': '47890177-f72f-4b94-85b0-d3f1d645fbee',
 'depth': 6,
 'scannedIn': '2020Q4',
 'distance': 585.55522363821}]

In [0]:
# Aggregate depths per quarter for the outlet
depth_df = pd.DataFrame(depth_result, columns = ['scannedIn','depth'])
depth_df = depth_df.groupby('scannedIn', as_index=False).agg({"depth": "mean"})
depth_df

Out[143]:

,scannedIn,depth
0,2020Q1,5.7
1,2020Q2,4.0
2,2020Q3,6.0
3,2020Q4,6.0


In [0]:
# Get Rain data logged that is within the range of the outlet
# outlet_item['location']['coordinates'][0] = 73.1117764
# outlet_item['location']['coordinates'][1] = 18.9093286
container_rain = database.create_container_if_not_exists(
    id="region_rain", 
    partition_key=PartitionKey(path="/city"),
)
distance_query = "ST_DISTANCE(f.boundary, {\"type\":\"Point\",\"coordinates\": [" + str(outlet_item['location']['coordinates'][0]) + ", " + str(outlet_item['location']['coordinates'][1]) + "]})"
rain_query = "SELECT f.id,f.depth,f.scannedIn,"+ distance_query +" as distance FROM f WHERE " + distance_query + " < 2000" 
rain_query

Out[139]: 'SELECT f.id,f.depth,f.scannedIn,ST_DISTANCE(f.boundary, {"type":"Point","coordinates": [73.7866902, 18.5558075]}) as distance FROM f WHERE ST_DISTANCE(f.boundary, {"type":"Point","coordinates": [73.7866902, 18.5558075]}) < 2000'

In [0]:
rain_result = list(container_rain.query_items(
    query=rain_query,
    enable_cross_partition_query=True
))
rain_result

Out[140]: []

In [0]:
# Aggregate rain per quarter for the outlet
rain_df = pd.DataFrame(rain_result, columns = ['scannedIn','rain'])
rain_df = rain_df.groupby('scannedIn', as_index=False).agg({"rain": "mean"})
rain_df

--------------------------------------------------------------------------- 
 DataError Traceback (most recent call last)
 <command-861169559008636> in <module> 
 1 # Aggregate rain per quarter for the outlet 
 2 rain_df = pd . DataFrame ( rain_result , columns = [ 'scannedIn' , 'depth' ] ) 
 ----> 3 rain_df = rain_df . groupby ( 'scannedIn' , as_index = False ) . agg ( { "depth" : "mean" } ) 
 4 rain_df

 /databricks/python/lib/python3.8/site-packages/pandas/core/groupby/generic.py in aggregate (self, func, engine, engine_kwargs, *args, **kwargs) 
 949 func = maybe_mangle_lambdas ( func ) 
 950 
 --> 951 result , how = self . _aggregate ( func , * args , ** kwargs ) 
 952 if how is None : 
 953 return result

 /databricks/python/lib/python3.8/site-packages/pandas/core/base.py in _aggregate (self, arg, *args, **kwargs) 
 414 
 415 try : 
 --> 416 result = _agg ( arg , _agg_1dim ) 
 417 except SpecificationError : 
 418 

 /databricks/python/lib/python3.8/site-packages/pandas/core/base.py in _agg (arg, func) 
 381 result = { } 
 382 for fname , agg_how in arg . items ( ) : 
 --> 383 result [ fname ] = func ( fname , agg_how ) 
 384 return result
 385 

 /databricks/python/lib/python3.8/site-packages/pandas/core/base.py in _agg_1dim (name, how, subset) 
 365 "nested dictionary is ambiguous in aggregation" 
 366 )
 --> 367 return colg . aggregate ( how ) 
 368 
 369 def _agg_2dim ( how ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/core/groupby/generic.py in aggregate (self, func, engine, engine_kwargs, *args, **kwargs) 
 238 
 239 if isinstance ( func , str ) : 
 --> 240 return getattr ( self , func ) ( * args , ** kwargs ) 
 241 
 242 elif isinstance ( func , abc . Iterable ) : 

 /databricks/python/lib/python3.8/site-packages/pandas/core/groupby/groupby.py in mean (self, numeric_only) 
 1391 Name : B , dtype : float64
 1392 """
 -> 1393 return self._cython_agg_general(
 1394 "mean" , 
 1395 alt = lambda x , axis : Series ( x ) . mean ( numeric_only = numeric_only ) , 

 /databricks/python/lib/python3.8/site-packages/pandas/core/groupby/groupby.py in _cython_agg_general (self, how, alt, numeric_only, min_count) 
 1049 
 1050 if len ( output ) == 0 : 
 -> 1051 raise DataError ( "No numeric types to aggregate" ) 
 1052 
 1053 return self . _wrap_aggregated_output ( output , index = self . grouper . result_index ) 

 DataError : No numeric types to aggregate

In [0]:
# Initialize X axis
x = pd.Series([1,2,3,4])
# Fetch Depth data for last 4 quarters
depth = pd.Series(depth_df['depth'])
# Fetch Rain data for last 4 quarters
rain = pd.Series(rain_df['rain'])

# Normalize depth and rain
depth = (depth - min(depth)) / (max(depth) - min(depth))
rain = (rain - min(rain)) / (max(rain) - min(rain))

# Calculate slope of the Depth metrics
depth_slope, intercept, r_value, p_value, std_err = linregress(x, depth)
depth_slope = depth_slope * -1
print("Depth slope:",depth_slope) # Invert depth slope as higher values mean lesser water

# Calculate slope of the Rain metrics
rain_slope, intercept, r_value, p_value, std_err = linregress(x, rain)
print("Rain slope:",rain_slope)

#Score is summation of both slopes
score = depth_slope + rain_slope
print("Score:",score)

Depth slope: 5.551115123125783e-18
Rain slope: -0.1
Score: -0.1

In [0]:
outlet_item['score'] = score
response = container.upsert_item(body=outlet_item)